In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [39]:
train = pd.read_csv("input/sales_train.csv")
test = pd.read_csv("input/test.csv")
items = pd.read_csv("input/items.csv")
itemsCat = pd.read_csv("input/item_categories.csv")
shops = pd.read_csv("input/shops.csv")
test["date_block_num"] = 34

In [40]:
import math
def get_second(strings):
    if isinstance(strings, str):
        temp = strings.split(" - ")
        if len(temp) > 1:
            return temp[1].strip()
        return "NA"
    return "NA"
def get_first(strings):
    if isinstance(strings, str):
        temp = strings.split(" - ")
        return temp[0].strip()
    return "NA"

In [41]:
test = (test
 .merge(items, left_on='item_id', right_on='item_id', how='inner')
 .sort_values("item_id")
 .merge(itemsCat, left_on='item_category_id', right_on='item_category_id', how='inner')
 .merge(shops, left_on='shop_id', right_on='shop_id', how='inner')
)

train = (train
 .merge(items, left_on='item_id', right_on='item_id', how='inner')
 .sort_values("item_id")
 .merge(itemsCat, left_on='item_category_id', right_on='item_category_id', how='inner')
 .merge(shops, left_on='shop_id', right_on='shop_id', how='inner')
)

test = test.dropna()
test["cat1"] = test.item_category_name.map(lambda x: x.split(" - ")[0].strip())
test["cat2"] = test.item_category_name.map(lambda x: get_second(x))

train = train.dropna()
train["cat1"] = train.item_category_name.map(lambda x: x.split(" - ")[0].strip())
train["cat2"] = train.item_category_name.map(lambda x: get_second(x))

In [45]:
train = train.drop(["item_name","item_category_name","shop_name","date","item_price"],axis=1,errors="ignore")
#train = train[["item_cnt_day","date_block_num","shop_id","item_id","item_category_id","cat1","cat2"]]
train = train[["item_cnt_day","date_block_num","shop_id","item_id","item_category_id"]]
train = train[(train["date_block_num"] == 22) | (train["date_block_num"] == 10)]
train.head()

,item_cnt_day,date_block_num,shop_id,item_id,item_category_id
2,1.000,22,54,2,40
13,1.000,22,54,11,40
16,1.000,22,54,15,40
75,1.000,10,54,30,40
76,1.000,10,54,30,40


In [46]:
test = test.drop(["item_name","item_category_name","shop_name"],axis=1,errors="ignore")
#test = test[["ID","date_block_num","shop_id","item_id","item_category_id","cat1","cat2"]]
test = test[["ID","date_block_num","shop_id","item_id","item_category_id"]]
test.head()

,ID,date_block_num,shop_id,item_id,item_category_id
0,206587,34,39,30,40
1,204595,34,39,32,40
2,204528,34,39,64,40
3,204596,34,39,67,40
4,207082,34,39,72,40


In [8]:
"""from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

def oneHotEncodeDF(someDF, numericalCols = None, categoricalCols = None):
    someDF[categoricalCols] = someDF[categoricalCols].apply(LabelEncoder().fit_transform)
    someDF[numericalCols] = someDF[numericalCols].apply(OneHotEncoder().fit_transform)
    someDF.drop(numericalCols,axis=1,inplace=True,errors="ignore")
    someDF.drop(categoricalCols,axis=1,inplace=True,errors="ignore")
    return someDF

cols_to_use = ["date_block_num","shop_id","item_id","item_category_id","cat1","cat2"]
X = train[cols_to_use]
y = train.item_cnt_day
train_X, test_X, train_y, test_y = train_test_split(X, y)"""

'from sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import OneHotEncoder\nfrom sklearn.preprocessing import LabelEncoder\n\ndef oneHotEncodeDF(someDF, numericalCols = None, categoricalCols = None):\n    someDF[categoricalCols] = someDF[categoricalCols].apply(LabelEncoder().fit_transform)\n    someDF[numericalCols] = someDF[numericalCols].apply(OneHotEncoder().fit_transform)\n    someDF.drop(numericalCols,axis=1,inplace=True,errors="ignore")\n    someDF.drop(categoricalCols,axis=1,inplace=True,errors="ignore")\n    return someDF\n\ncols_to_use = ["date_block_num","shop_id","item_id","item_category_id","cat1","cat2"]\nX = train[cols_to_use]\ny = train.item_cnt_day\ntrain_X, test_X, train_y, test_y = train_test_split(X, y)'

In [50]:
len(train)

183164

In [51]:
colsToTransform = ["shop_id","item_id","item_category_id"]
allData = pd.get_dummies(train.append(test),columns = colsToTransform)
train = allData[:len(train)]
test = allData[len(train):]

In [54]:
test

,ID,date_block_num,item_cnt_day,shop_id_56.0,shop_id_57.0,shop_id_58.0,shop_id_59.0,shop_id_61.0,shop_id_62.0,shop_id_64.0,...,item_category_id_142.0,item_category_id_143.0,item_category_id_144.0,item_category_id_145.0,item_category_id_146.0,item_category_id_147.0,item_category_id_148.0,item_category_id_150.0,item_category_id_151.0,item_category_id_155.0
183164,nan,nan,nan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183165,nan,nan,nan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183166,nan,nan,nan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183167,nan,nan,nan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183168,nan,nan,nan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183169,nan,nan,nan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183170,nan,nan,nan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183171,nan,nan,nan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183172,nan,nan,nan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183173,nan,nan,nan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#cols_to_use = ["date_block_num","shop_id","item_id","item_category_id","cat1","cat2"]
from sklearn.model_selection import train_test_split
X = train.loc[:, train.columns != 'item_cnt_day']
y = train.item_cnt_day
train_X, test_X, train_y, test_y = train_test_split(X, y)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline

my_pipeline = make_pipeline(RandomForestRegressor(verbose=True))

In [16]:
my_pipeline.fit(train_X, train_y)
predictions = my_pipeline.predict(test_X)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 22.6min finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.3s finished


In [17]:
predictions

array([ 1.       ,  4.2724026,  1.       , ...,  1.       ,  1.       ,  1.       ])

In [18]:
ans_predictions = my_pipeline.predict(test.loc[:, test.columns != 'ID'])

ValueError: Number of features of the model must match the input. Model n_features is 6494 and input n_features is 5269 